# Twitter data extraction and storing in a dataset

In [44]:
import tweepy
import json
import pandas as pd
from scipy.misc import imread
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib as mpl
import csv
import matplotlib.pyplot as plt


# Authentication

In [45]:
consumer_key = 'bb32Oh0hhUDz1IVOSJh5QTa89'
consumer_secret = 'ZkU2kYfFqiyT2dJKtg21QROCgy9oMPzgCsmjbyZNB9yQAEDtcn'
access_token = '1069450930045812736-zTW0GusPltHRX8ljuLQXsMgicS7xpi'
access_token_secret = 'tvdh7qXEBcWG5lPyri0jCrfaz0Yji555Gqv7SOIoT71Mt'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) #Interacting with twitter's API
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API (auth) #creating the API object


#Extracting Tweets
results = []
for tweet in tweepy.Cursor (api.search, q = 'millennials', lang = "en").items(2000): 
    results.append(tweet)
    
print (type(results))
print (len(results))

<class 'list'>
2000


In [46]:
def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"])
    
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]
    
    return data_set
data_set = tweets_df(results)

# Finding the things which were asked

In [47]:
data_set


,id,text,created_at,retweet_count,user_screen_name,user_followers_count,user_location,Hashtags
0,1102237450045452289,RT @MilleDescubren: Millennials be like: https...,2019-03-03 16:01:01,17,ULLOAHERNAN,511,"Barranquilla, Colombia",[]
1,1102237313151561728,RT @TheDemocrats: While Betsy DeVos continues ...,2019-03-03 16:00:28,156,willmdashw,185,Portland Oregon,[]
2,1102237282415902721,"Divine Risings! #IMakeDopeMusic, #SnackDaddy a...",2019-03-03 16:00:21,0,boogie_nightz_,1381,Houston - Atlanta,"[{'text': 'IMakeDopeMusic', 'indices': [16, 31..."
3,1102237277487579142,Why #Millennials are viewed the way they are s...,2019-03-03 16:00:20,0,TTBL22,462,,"[{'text': 'Millennials', 'indices': [4, 16]}]"
4,1102237247984857089,RT @MilleDescubren: Millennials be like: https...,2019-03-03 16:00:13,17,MaxoLlevant,790,Ciutat de València,[]
5,1102237245183086592,When it comes to music recognition of older so...,2019-03-03 16:00:12,0,MusicalityRobin,497,"Seattle, WA","[{'text': 'millennials', 'indices': [84, 96]}]"
6,1102237237201256449,Busting The Myth Of The Workshy Millennial htt...,2019-03-03 16:00:10,0,lindahobbis,13702,United Kingdom,"[{'text': 'bloggers', 'indices': [67, 76]}, {'..."
7,1102237233191489536,When it comes to music recognition of older so...,2019-03-03 16:00:09,0,SacMusicAcademy,83,"Sacramento, California","[{'text': 'millennials', 'indices': [84, 96]}]"
8,1102237217244827648,Why aren't millennials buying blunderbusses?,2019-03-03 16:00:05,0,WhyMillennials,76,"London, England",[]
9,1102237205735587840,RT @robsmithonline: I had fun chatting with @f...,2019-03-03 16:00:03,62,morrowfcs,2328,,[]


In [48]:
print(len(results))    #1.total number of tweets
l=[]
for i in data_set['user_screen_name']:
    l.append(i)
s= list(set(l)) #2.unique users
f=[] #count of tweets per unique user
for d in s:
    e=l.count(d)
    f.append(e)

Z = [X for _,X in sorted(zip(f,s),reverse=True)]
print(Z[0:10]) #3.top users with maximum tweets
    
    
    
print(len(s))    #unique users

2000
['boomerpanicbot', 'WhyMillennials', 'millenialsbot', 'GillemKip', 'millennials_bot', 'NancyRog11', 'Jennyjinx', 'Bad_Millennials', 'theuppitynegro', 'millennial_kill']
1810


In [49]:
m=[]  #list of retweet count
for k in data_set['retweet_count']:
    m.append(k)
n=[]    
h=[]
for j in range(len(data_set['text'])):
    if data_set['retweet_count'].iloc[j]==max(m):
        n.append(data_set['text'])
        h.append(data_set['user_screen_name'])
        
print(len(n)) #4.the number of tweets with maximum retweets
print(h) #5.users with maximum retweets

1
[0           ULLOAHERNAN
1            willmdashw
2        boogie_nightz_
3                TTBL22
4           MaxoLlevant
5       MusicalityRobin
6           lindahobbis
7       SacMusicAcademy
8        WhyMillennials
9             morrowfcs
10         HardiVark_64
11        SexyLilToyBox
12            trebor199
13            jazzpromo
14          enigmasept9
15              edsalag
16         kimmilouhoo1
17      Pensardiferente
18      OverUnderClover
19       spatel16333289
20      angelsfabulous1
21             Hubertes
22           AnaPerezas
23          iBrianJones
24               upuouo
25            JohnNoizz
26              trb2997
27         plangeypants
28           screaltor1
29            NightEvE_
             ...       
1970       fucsiabird77
1971          sy_medusa
1972          heistness
1973      shahin_gourgi
1974     itamarislguide
1975          tradnews_
1976    nytimesbusiness
1977       Annie__Storm
1978          elly_army
1979         jhnmrfrmlj
1980         

In [50]:
q=[]
o= list(set(m))
for p in o:
    a= m.count(p)
    q.append(a)
print(max(q))

for r in range(len(m)):
    if m.count(m[r])==max(q):
        print(data_set['text'].iloc[r])  #6.tweets with the most favourite count of retweets
        

772
Divine Risings! #IMakeDopeMusic, #SnackDaddy and #SnackMommy t-shirts now available at https://t.co/zb1zaoukFx… https://t.co/FhPaB38sfZ
Why #Millennials are viewed the way they are summed up in one headline. https://t.co/SPMNs02BW1
When it comes to music recognition of older songs, give credit where credit is due. #millennials https://t.co/gXIum1PsBw
Busting The Myth Of The Workshy Millennial https://t.co/0TKSAaAJv5 #bloggers #millennials https://t.co/OSM5iLD8PD
When it comes to music recognition of older songs, give credit where credit is due. #millennials https://t.co/aqzn1rkLya
Why aren't millennials buying blunderbusses?
@ARedPillReport Bernie's sick twisted socialist ass is pandering to every group.

Mexicans and illegals by hiring a… https://t.co/Dmt4eK2Keq
Jack Kleinsinger's Highlights Presents Satchmo and Sidney Revisited Thursday, March 28th 8 PM BMCC TRIBECA Performi… https://t.co/RZPr20y2PO
@JamilahLemieux I think some millennials aren't anxious to buy a house because th

According to this, millennials are buying and seniors are selling or renting.  Whatever is the path for you, give C… https://t.co/zEhzw7LD8I
The beauty of this article is it applies to more than millennials. High achieving women in the corporate world easi… https://t.co/9JDLSuBkWT
Millennials https://t.co/SCMEgb6bOB
@Adam_Karpiak My prediction ; Mischief is going to be in top 5 of what millennials are seeking in workplace articles on trend
“Homeowners are worth 40 times more than renters. Now, that first home doesn’t need to be a dream home, it can be a… https://t.co/UM63mUbtbC
On this day millennials murdered cupola at the yoga retreat with kale. RIP cupola.
Baby boomers better be nicer to us Millennials, next time there’s an unexpected item in the bagging area you’re fucked.
64% of Millennials say they are still pretty brand loyal.  https://t.co/RjU554MFO4
Over one hundred million people make up #Millennials and #GenerationZ. How do we motivate these young people? How d… https://t.co

I see no millennials but parents of millennials here😓 https://t.co/hbMkgtZL6W
@heyriddleriddle Riddy Kitty is the best. I would say better than any of the shitty normal hosts of Hello Puzzle Pu… https://t.co/v3KLm6R0nK
@john_c_palmer This has “Millennials in the workforce” written all over it. The only trouble would be training peop… https://t.co/3fb1WxVc5s
"What cannabis can learn from Glossier." Blog post coming soon! 

"72% of millennials take product recommendations… https://t.co/19XhYDyEyQ
Gen X might be the greatest generation of the 20th century.  Two tech booms, lots of nice jobs, and ride the Boomer… https://t.co/8oSNnkpAvb
Then don’t take basket weaving  classes that are unemployable! https://t.co/fu9tUvlKwk
Why aren't millennials buying sorgos?
“Mainstream consumer adoption of bitcoin” is the “Linux on desktop” for millennials.
Many things that are self evident have to be restated now.  IQs and EQs of millennials and generations xyz are at a… https://t.co/CWb2ndLXFZ
@HuffPos

"Creative Director" is the new imaginary job role for Millennials
wow, Joy is so willfully out of touch that she ar least claims to believe that millennials just love them some cree… https://t.co/m1LZXEIlxy
#Millennials Face $1 Trillion Debt as Student Loans Pile Up

STOP GOING TO COLLEGE
GO TO TRADE $CHOOL https://t.co/45ZiNP9Llm
Millennials have voted Alexis Ren hottest on internet (35 Photos) https://t.co/mlRqrwIWk1
@cevangelista413 Exactly. Living in NYC, I’m lucky that there are still top-notch movie houses but the multiplex ex… https://t.co/mnjPPWlupS
Leonard and Hungry Paul review: Dorks in a time of pithy millennials https://t.co/DQw4L6zW3A
oh wait i totally forgot to talk about the weirdest moment i had while at work today lmao:
me: hey how's your day g… https://t.co/NVvBAyHBZA
Damn. Millennials are Killing the US Beer Industry &amp;#8211; Their Next Victim is Heineken https://t.co/mLyFjUA7F8
most people don't really know what year millennials were born and just give everyone 

"majority of millennials today are having difficulty saving up for their future, and reality is that many are trapp… https://t.co/N0rRHrECJu
Millennials ruined sonata caravan
Ottawa wants to help millennials to buy homes, RBC says there is no ‘magic bullet’ https://t.co/8fEgRF7HX3
a good question here....why are some millennials turning to the supernatural?

https://t.co/msMkSvh7Rs
Help! Millennials Held Up Our Golf Bag !!
Ottawa wants to help millennials to buy homes, RBC says there is no ‘magic bullet’ https://t.co/cElyC2ge6c
@qandrew Weirdos, millennials, and @PETA followers. Who wouldn't pet her/him/it/that/cat ?!
@amybruni Eat it safely? Lol I have no clue. Millennials.😂
@MrAlexOrtegaaa @educationweek With all the problems facing public education driving “super far” is silly thing to… https://t.co/CMhAd2fWkC
Why aren't millennials buying male bondings?
Types of Annoying Millennials https://t.co/3Pu46AsON0 via @YouTube
More welcoming immigration policies in Canada and the recent de

In [51]:
u=[] #all locations
for t in data_set['user_location']:
    u.append(t)
v= list(set(u)) #unique locations
x= [] # count of each unique location
for w in v:
    b= u.count(w)
    x.append(b)

for y in range(len(v)):
    print(v[y],x[y]) #7.location and the count of each location
    
    

 565
S.E. Queensland, Australia 1
England, United Kingdom 2
Amsterdam, The Netherlands 1
The South.... 1
Marana, AZ 1
NYC 3
www.astoldbyqueenie.tumblr.com 1
Sunny California 1
Busy being a easy, breezy, beautiful, bad Bitch 1
Manhattan 10040 1
London / DC 1
No 12, Grimmauld Place 1
England 1
Harare, Zimbabwe 1
nc | pa | online 1
Berkeley, CA 1
KL, MY 1
Delray Beach, FL and Atlantis, FL 1
Indonesia 2
United States  1
Finland 1
Cherry Hill, NJ 1
West Village, Manhattan 1
Iraq 1
abandoned airport 1
Los Angeles 2
Hamilton, Ontario 1
🇬🇹 1
Illinois 2
Everwhere, USA  1
Gainesville, FL 1
Tennessee 1
Fort Area, Mumbai 2
Mustafar 1
Boston, Mass 1
Salt Lake City, UT 1
Nashville, TN 2
Phoenix, Arizona currently 1
wakanda 1
Lucknow, India 1
Los Angeles & San Diego, Ca  1
South Africa, Johannesburg 1
Bangalore 2
NY 1
Tanzania, East Africa. 1
Sterling VA 1
Brevard & Volusia counties, FL 2
MADRID / LONDRES 1
Scottsdale, AZ 1
Ankeny, Iowa 1
Gingoog City,Northern Mindanao 1
Twitter 1
Wiltshire;Wales; So

 Wis. U.S 1
London, UK 1
Planet Earth 1
New Jersey, USA 4
Manila 1
her own space 1
Pahang, Malaysia 1
Memphis, TN 2
Pulau pinang 1
Ealing, London 2
Hong Kong 1
Jecha 1
USA TODAY 1
Toronto (Brampton) - Canada 1
inna cove in da hood.  1
Fredros_Inc; Planet Earth; Ifa 1
New York,NY 1
London/Ramallah/Gaza/Vilna 1
melb 1
Санкт-Петербург, Россия 1
she/her 1
Penang, Malaysia 1
Seremban, Negeri Sembilan 2
The Abyss 1
nyc/long island  1
Bangladesh 1
Cancún, Mexico 1
the toxic waste dumps of nj 1
Paris, France 2
Tallahassee, FL 1
Orygun 1
Dublin, Ireland 1
Bogotá 1
California Girl 1
Manhattan, NY 1
Brooklyn, NY 10
285 lbs to 216.8 ⬇️ 1
low key army 🤪 1
NY State 2
Wallkill, NY, US, 12589 1
Washington, DC (formerly Pittsburgh & Michigan. Not from or of NY.) 1
Jupiter's Sulfur Mines 1
Cleveland, Oh 1
Hebden Bridge, UK 1
Rocket Center, WV 1
World 1
Oxnard, CA 1
DC 1
North Carolina,OBX 1
West Philly Baby 1
The Diamond District 1
Georgia USA 1
Hollywood, Los Angeles 1
Valdosta, GA 1
клуб ростокинских 

In [59]:
import plotly
plotly.tools.set_credentials_file(username='ruchi_jangir', api_key='K3cBmsRTJ0H4VE04GkGu')
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF

In [62]:


data = [go.Scatter(x = data_set['user_followers_count'],y = data_set['retweet_count'],mode = 'markers')]

layout = go.Layout(title='retweet_count vs user_followers_count',
xaxis=dict(title='user_followers_count'),
yaxis=dict(title='retweet_count'))

fig = go.Figure(data=data, layout=layout)

py.iplot(fig)